# Pythonのロギングの仕組みを理解する

勉強会の資料用Notebookです。


## Loggerを使う

In [ ]:
from logging import getLogger

# Setup logger
logger = getLogger(__name__)

python の Logger はこうやって初期化するのが一般的です。
あとで解説するので `__name__` とかはおまじないだと思ってください。  

早速このloggerを使ってみましょう。

In [ ]:
from logging import getLogger

logger = getLogger(__name__)

def main():
    logger.debug('This is a Debug message')
    logger.info('This is a Info message')
    logger.warning('This is a Warning message')
    logger.error('This is a Error message')
    logger.critical('This is a Critical message')
    
if __name__ == '__main__':
    main()

他の言語にもあるように Python でも `debug` `info` `warning` `error` `critical` の6種類のログレベルが使用できます。
特に難しいことはないですね。

実行結果も見てみましょう。

>This is a Warning message
>This is a Error message
>This is a Critical message

あれ`debug`,` info` はどこへ…。

In [ ]:
from logging import getLogger, DEBUG

logger = getLogger(__name__)

# これかな?
logger.setLevel(DEBUG)


def main():
    logger.debug('This is a Debug message')
    logger.info('This is a Info message')
    logger.warning('This is a Warning message')
    logger.error('This is a Error message')
    logger.critical('This is a Critical message')
    
if __name__ == '__main__':
    main()

`logger` には `setLevel` という、どのレベル以上のものを表示するかという設定があります。
これで `info` や `debug` が無視されていたに違いないので、早速 `DEBUG` 以上をセットしてみましょう。

>This is a Warning message
>This is a Error message
>This is a Critical message

ぐぬぬ…

In [2]:
import sys
from logging import getLogger, DEBUG, StreamHandler

logger = getLogger(__name__)
logger.setLevel(DEBUG)

#  これも？
handler = StreamHandler(sys.stderr)
handler.setLevel(DEBUG)
logger.addHandler(handler)


def main():
    logger.debug('This is a Debug message')
    logger.info('This is a Info message')
    logger.warning('This is a Warning message')
    logger.error('This is a Error message')
    logger.critical('This is a Critical message')
    
if __name__ == '__main__':
    main()

This is a Debug message
This is a Info message
This is a Warning message
This is a Error message
This is a Critical message


Python の Logger には Handler という概念があります。
Handler というのは書き込まれたログをどう扱うかを決めるためのものです。
デフォルトだと標準エラー出力にログが表示されるだけですが、これをカスタマイズすることで、ファイルに書き込むようにしたり、出力のフォーマットや要素を好きなように変更することが出来ます。
この Handler も ログレベルのしきい値を持っているので、こっちのレベルも変更してみましょう。

>This is a Debug message
>This is a Info message
>This is a Warning message
>This is a Error message
>This is a Critical message

👏
無事全部のログが出せましたね。

## Logger について

## Handler について

## Formatter について

## スッキリ書く